In [1]:
import numpy as np
import igraph as ig
from scipy.special import betaln

Data Generation

In [39]:
theta = np.array([[0.75, 0.25, 0.1, 0.9],
                  [0.1, 0.75, 0.1, 0.1],
                  [0.1, 0.75, 0.25, 0.9]])

In [40]:
Sr = [20, 20, 20]
Sc = [15, 15, 15, 15]

In [41]:
def generate_graph_bicluster(Sr, Sc, thetas):
    # Kr = thetas.shape[0]
    # Kc = thetas.shape[1]

    N = np.sum(Sr)

    Hr = []
    c = 0
    for i in Sr:
        for j in range(i):
            Hr += [c]
        c+=1

    Hc = []
    c = 0
    for i in Sc:
        for j in range(i):
            Hc += [c]
        c+=1

    X = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            X[i, j] = np.random.binomial(1, thetas[Hr[i], Hc[j]])

    return X


In [42]:
X = generate_graph_bicluster(Sr, Sc, theta)

In [43]:
def irm_bicluster(X, T, a, b, A, set_seed = True, random_seed = 42, print_iter = False):
    N = len(X)
    zr = np.ones([N, 1])
    zc = np.ones([N, 1])
    Z = []
    Zr = []
    Zc = []

    if set_seed:
        np.random.seed(random_seed)

    for t in range(T):
        for n in range(N):
            nn = [_ for _ in range(N)]
            nn.remove(n)

            X_ = X.copy().astype(int)
            X_[n,:] = 0

            Kr = len(zr[0])

            if Kr > 1:
                idx = np.argwhere(np.sum(zr[nn], 0) == 0)
                zr = np.delete(zr, idx, axis=1)
                Kr -= len(idx)

            # m = n. of nodes in each component 
            mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
            mc = np.sum(zc[nn,:], 0)[np.newaxis]
            Mc = np.tile(mc, (Kr, 1))
            M1 = zr.T @ X_ @ zc

            X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
            X_rev[n,:] = 0
            M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

            r = zc[nn,:].T @ X[n, nn]
            R = np.tile(r, (Kr, 1))

            logLik_exComp = np.sum(betaln(M1+R+a, M0+Mc-R+b) - betaln(M1+a, M0+b),1)
            logLik_newComp = np.sum(betaln(r+a, mc-r+b) - betaln(a,b),1)

            logLik = np.concatenate([logLik_exComp, logLik_newComp])
            logPrior = np.log(np.append(mr, A))

            logPost = logPrior + logLik

            P = np.exp(logPost-max(logPost)) 

            # Assignment through random draw fron unif(0,1), taking first value from prob. vector
            draw = np.random.rand()
            i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

            zr[n,:] = 0
            if i == Kr: # If new component: add new column to partition matrix
                zr = np.hstack((zr, np.zeros((N,1)))) 
            zr[n,i] = 1

        for n in range(N):
            nn = [_ for _ in range(N)]
            nn.remove(n)

            X_ = X.copy().astype(int)
            X_[:,n] = 0

            Kc = len(zc[0])

            if Kc > 1:
                idx = np.argwhere(np.sum(zc[nn], 0) == 0)
                zc = np.delete(zc, idx, axis=1)
                Kc -= len(idx)

            # m = n. of nodes in each component 
            mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
            mc = np.sum(zc[nn,:], 0)[np.newaxis]
            Mr = np.tile(mr.T, (1, Kc))

            M1 = zr.T @ X_ @ zc

            X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
            X_rev[:,0] = 0
            M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

            s = zr[nn,:].T @ X[nn, n]
            S = np.tile(s[np.newaxis].T, (1, Kc))

            logLik_exComp = np.sum(betaln(M1+S+a, M0+Mr-S+b) - betaln(M1+a, M0+b), 0)
            logLik_newComp = np.sum(betaln(s+a, mr-s+b) - betaln(a,b),1)

            logLik = np.concatenate([logLik_exComp, logLik_newComp])
            logPrior = np.log(np.append(mc, A))

            logPost = logPrior + logLik

            P = np.exp(logPost-max(logPost)) 

            # Assignment through random draw fron unif(0,1), taking first value from prob. vector
            draw = np.random.rand()
            i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

            zc[n,:] = 0
            if i == Kc: # If new component: add new column to partition matrix
                zc = np.hstack((zc, np.zeros((N,1)))) 
            zc[n,i] = 1

        Z.append([zr.copy(), zc.copy()])
    
    return Z

In [44]:
X

array([[0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [45]:
T = 500
a = 1
b = 1
A = 1

In [46]:
Z = irm_bicluster(X, T , a, b, A, set_seed = True, random_seed = 42)

In [47]:
t = Z[-1]

In [48]:
t

[array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1

In [264]:
#test X
X = np.array([[0, 1, 0, 0, 1],
              [0, 0, 1, 1, 0],
              [0, 1, 0, 1, 1],
              [1, 1, 0, 0, 1],
              [0, 1, 0, 1, 0]])

zr = np.array([[1, 0],
               [1, 0],
               [1, 0],
               [0, 1],
               [0, 1]])

zc = np.array([[1, 0, 0],
               [1, 0, 0],
               [0, 1, 0],
               [0, 1, 0],
               [0, 0, 1]])

In [32]:
N = len(X)
# zr = np.ones([N, 1])
# zc = np.ones([N, 1])
Zr = []
Zc = []

In [55]:
# for t in range(T):
#     for n in range(N):
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

X_ = X[np.ix_(nn,nn)]

Kr = len(zr[0])
Kc = len(zc[0])

if Kr > 1:
    idx = np.argwhere(np.sum(zr[nn], 0) == 0)
    zr = np.delete(zr, idx, axis=1)
    Kr -= len(idx)

# m = n. of nodes in each component 
mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
mc = np.sum(zc[nn,:], 0)[np.newaxis]

# M = max links from other clusts
# M = np.tile(m, (Kr, 1))

M1 = zr[nn,:].T @ X_ @ zc[nn,:]

X_rev = np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0]) #reverse matrix for non_links
M0 = zr[nn,:].T @ X_rev @ zc[nn,:] #n. of non-links between biclusters without current node

r = zc[nn,:].T @ X[n, nn]
R = np.tile(r, (Kr, 1))

s = zr[nn,:].T @ X[nn, n]
S = np.tile(s[np.newaxis].T, (1, Kc))



In [162]:
# for t in range(T):
#     for n in range(N):
a = 1
b = 1
A = 1

N = len(X)
for i in range(N):
    nn = [_ for _ in range(N)]
    nn.remove(n)

    # X_ = X[np.ix_(nn,nn)]

    X_ = X.copy()
    X_[n,:] = 0

    Kr = len(zr[0])
    Kc = len(zc[0])

    if Kr > 1:
        idx = np.argwhere(np.sum(zr[nn], 0) == 0)
        zr = np.delete(zr, idx, axis=1)
        Kr -= len(idx)

    # m = n. of nodes in each component 
    mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
    mc = np.sum(zc[nn,:], 0)[np.newaxis]
    Mc = np.tile(mc, (Kr, 1))

    # M = max links from other clusts
    # M = np.tile(m, (Kr, 1))

    M1 = zr.T @ X_ @ zc

    X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
    X_rev[n,:] = 0
    M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

    r = zc[nn,:].T @ X[n, nn]
    R = np.tile(r, (Kr, 1))

    logLik_exComp = np.sum(betaln(M1+R+a, M0+Mc-R+b) - betaln(M1+a, M0+b),1)
    logLik_newComp = np.sum(betaln(r+a, mc-r+b) - betaln(a,b),1)

    logLik = np.concatenate([logLik_exComp, logLik_newcomp])
    logPrior = np.log(np.append(mr, A))

    logPost = logPrior + logLik

    P = np.exp(logPost-max(logPost)) 

    # Assignment through random draw fron unif(0,1), taking first value from prob. vector
    draw = np.random.rand()
    i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

    zr[n,:] = 0
    if i == Kr: # If new component: add new column to partition matrix
        zr = np.hstack((zr, np.zeros((N,1)))) 
    zr[n,i] = 1


In [16]:
N = len(X)

zr = np.ones([N, 1])
zc = np.ones([N, 1])

In [17]:
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

# X_ = X[np.ix_(nn,nn)]

X_ = X.copy()
X_[:,n] = 0

Kr = len(zr[0])
Kc = len(zc[0])

In [20]:
mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
mc = np.sum(zc[nn,:], 0)[np.newaxis]
Mc = np.tile(mc, (Kr, 1))

M1 = zr.T @ X_ @ zc

In [32]:
X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy()

In [31]:
X_ = X_.astype(int)

In [33]:
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

# X_ = X[np.ix_(nn,nn)]

X_ = X.copy().astype(int)
X_[:,n] = 0

Kr = len(zr[0])
Kc = len(zc[0])

if Kr > 1:
    idx = np.argwhere(np.sum(zr[nn], 0) == 0)
    zr = np.delete(zr, idx, axis=1)
    Kr -= len(idx)

# m = n. of nodes in each component 
mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
mc = np.sum(zc[nn,:], 0)[np.newaxis]
Mc = np.tile(mc, (Kr, 1))

# M = max links from other clusts
# M = np.tile(m, (Kr, 1))

M1 = zr.T @ X_ @ zc

X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
X_rev[n,:] = 0
M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

r = zc[nn,:].T @ X[n, nn]
R = np.tile(r, (Kr, 1))

logLik_exComp = np.sum(betaln(M1+R+a, M0+Mc-R+b) - betaln(M1+a, M0+b),1)
logLik_newComp = np.sum(betaln(r+a, mc-r+b) - betaln(a,b),1)

logLik = np.concatenate([logLik_exComp, logLik_newComp])
logPrior = np.log(np.append(mr, A))

logPost = logPrior + logLik

P = np.exp(logPost-max(logPost)) 

# Assignment through random draw fron unif(0,1), taking first value from prob. vector
draw = np.random.rand()
i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

zr[n,:] = 0
if i == Kr: # If new component: add new column to partition matrix
    zr = np.hstack((zr, np.zeros((N,1)))) 
zr[n,i] = 1

NameError: name 'logLik_newcomp' is not defined

In [297]:
for n in range(N):
    nn = [_ for _ in range(N)]
    nn.remove(n)

    # X_ = X[np.ix_(nn,nn)]

    X_ = X.copy()
    X_[:,n] = 0

    Kr = len(zr[0])
    Kc = len(zc[0])

    if Kc > 1:
        idx = np.argwhere(np.sum(zc[nn], 0) == 0)
        zc = np.delete(zc, idx, axis=1)
        Kc -= len(idx)

    # m = n. of nodes in each component 
    mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
    mc = np.sum(zc[nn,:], 0)[np.newaxis]
    Mr = np.tile(mr.T, (1, Kc))

    M1 = zr.T @ X_ @ zc

    X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
    X_rev[:,0] = 0
    M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

    s = zr[nn,:].T @ X[nn, n]
    S = np.tile(s[np.newaxis].T, (1, Kc))

    logLik_exComp = np.sum(betaln(M1+S+a, M0+Mr-S+b) - betaln(M1+a, M0+b), 0)
    logLik_newComp = np.sum(betaln(s+a, mr-s+b) - betaln(a,b),1)

    logLik = np.concatenate([logLik_exComp, logLik_newComp])
    logPrior = np.log(np.append(mc, A))

    logPost = logPrior + logLik

    P = np.exp(logPost-max(logPost)) 

    # Assignment through random draw fron unif(0,1), taking first value from prob. vector
    draw = np.random.rand()
    i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

    zc[n,:] = 0
    if i == Kc: # If new component: add new column to partition matrix
        zc = np.hstack((zc, np.zeros((N,1)))) 
    zc[n,i] = 1

In [298]:
zc

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [268]:
np.sum(betaln(M1+S+a, M0+Mr-S+b) - betaln(M1+a, M0+b), 0)

array([-4.19970508, -3.14988295, -3.40119738])

In [271]:
np.sum(betaln(s+a, mr-s+b) - betaln(a,b), 1)

array([-2.89037176])

In [289]:
zc[n,:] = 0
if i == Kc: # If new component: add new column to partition matrix
    zc = np.hstack((zc, np.zeros((N,1)))) 
zc[n,i] = 1

In [290]:
zc

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])